# Logging in Jupyter Notebooks with IPython

Logging is a happy thing. Let's learn how to do professional quality logging. How to properly do it with Jupyter Notebooks, and IPython? [The general Python logging infrastructure is described here](https://docs.python.org/3/library/logging.html), and is implemented by the `logging` module.

![](https://upload.wikimedia.org/wikipedia/commons/8/8a/H96566k.jpg "The first bug, Courtesy of the Naval Surface Warfare Center, Dahlgren, VA., 1988. - U.S. Naval Historical Center Online Library Photograph NH 96566-KN, Public Domain, https://commons.wikimedia.org/w/index.php?curid=165211")

## First, IPython magic (⊃｡•́‿•̀｡)⊃━☆ﾟ.*･｡ﾟ

IPython offers some [magic for logging](http://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-logstart). I don't yet have any idea if this is about logging the IPython environment itself, or is this also useful for logging application development.

* `%logstart`: activates logging to a file
* `%logstop`: stops the above
* `%logoff`: for temporarily suspending logging after it has been started
* `%logon`: for resuming suspended logging
* `%logstate`: queries the status

In [13]:
%logstart

Activating auto-logging. Current session state plus future input saved.
Filename       : ipython_log.py
Mode           : rotate
Output logging : False
Raw input log  : False
Timestamping   : False
State          : active


In [14]:
%logstate

Filename       : ipython_log.py
Mode           : rotate
Output logging : False
Raw input log  : False
Timestamping   : False
State          : active


In [15]:
%logstop

I'll try some new style print formatting, while i'm on it.

In [32]:
with open('ipython_log.py') as fd:
    loglines = fd.readlines()
    
print("Log contains total of {n_lines} lines, of which {n_unique} are unique".format(n_lines=len(loglines), n_unique=len(set(loglines))))
for line in set(loglines):
    print("{count:4}: {l}".format(count=loglines.count(line), l=line.strip()))

Log contains total of 17 lines, of which 7 are unique
   1: get_ipython().magic('logon')
   1: get_ipython().magic('logend')
   4: get_ipython().magic('logstop')
   3: get_ipython().magic('logstart')
   1: # IPython log file
   1: 
   6: get_ipython().magic('logstate')


## Normal Python logging

Anyhow, in Python the typical logging capabilities are provided by the `logging` module.

In [36]:
import logging

Now,

In [37]:
logging.info("hello")

does not *seem* to do anything, because IPython already has started logging, and called `logging.basicConfig`. Let's get a reference to it.

In [58]:
logger = logging.getLogger()
logger.__dict__ # standard Python object description

{'disabled': False,
 'filters': [],
 'handlers': [<logging.StreamHandler at 0x7f2592ba5390>],
 'level': 30,
 'name': 'root',
 'parent': None,
 'propagate': True}

In [64]:
logger.handlers[0].__dict__

{'_name': None,
 'filters': [],
 'formatter': <logging.Formatter at 0x7f2591dbaf98>,
 'level': 0,
 'lock': <unlocked _thread.RLock object owner=0 count=0 at 0x7f2591de6cf0>,
 'stream': <_io.TextIOWrapper name='<stderr>' mode='w' encoding='UTF-8'>}

Ok so that goes to stderr. The default `root` logger defaults to level 30.

The following should be visible on the console from where IPython was conjured, or wherever this IPython happens to send it's standard error, because level of `logger.critical` is strictly higher than what we have at the root logger level (ie. >30).

In [66]:
logger.critical('hello')

And indeed it does!

![](a critical log message.png)

From the [`logging` module documentation](https://docs.python.org/3/library/logging.html#levels) we learn that the logging levels are:

|Level    |Numeric value|
|---------|------------:|
|CRITICAL |50|
|ERROR    |40|
|WARNING  |30|
|INFO     |20|
|DEBUG    |10|
|NOTSET   |0|

Ok, so the logger called *root* has been defined, and is accessible by getting it. I believe what I want is to create my own logger.

In [68]:
mylogger = logging.getLogger('mylogger')
mylogger.__dict__

{'disabled': False,
 'filters': [],
 'handlers': [],
 'level': 0,
 'manager': <logging.Manager at 0x7f25975da2b0>,
 'name': 'mylogger',
 'parent': <logging.RootLogger at 0x7f25975da240>,
 'propagate': True}

In [72]:
mylogger.setLevel(logging.INFO)
mylogger.info('I is I')

## Integrating to workflows

Great, a new logger was greated and it does log into the console.

Now, how to tie it to the literate programming Notebook workflow? [Having a root logger is reported as an issue on IPython GitHub](https://github.com/ipython/ipython/issues/8282), which is weird since I think it's ok to have a logger. The logger framework is quite a construction on it's own.

One idea is to set the logging level of the default logger when doing developing work, like so:

In [81]:
def workit(i):
    for i in range(0, i):
        logging.debug("Starting a costly computational operation, i={}".format(i))
        print(i, end=' ')

In [84]:
logging.getLogger().setLevel(logging.DEBUG)
workit(5)

0 1 2 3 4 

The above does print to the log. And then when done with developing, set

In [85]:
logging.getLogger().setLevel(logging.WARNING)
workit(2)

0 1 

Two ideas were tried above: getting a new logger which defaults to level `logging.NOTSET` = 0 and using it for all logging purposes, or getting the previously existing root logger and adjusting it's level. Both seem to work. The former has the advantage that the logging messages bear a more specific name than "root".

What one would like to do further is adjust log handlers for creating f.ex. logfiles, and somehow getting logging messages into the Notebooks perhaps? Is this a useful workflow, for f.ex. data analysis jobs? On one hand logging messages are clutter, but on the other hand one want's to do literate programming. Hmm perhaps level `INFO` should come to the Notebook, and `DEBUG` should only go to the console in the background. That sounds like setting up a logging handler for `INFO` messages.